# Configs

`.. currentmodule:: lab.configs`

In [1]:
import torch

from lab import tracker, monit, loop, experiment, logger
from lab.configs import BaseConfigs

The configs will be stored and in future be adjusted from  [Dashboard](https://github.com/vpj/lab_dashboard)

In [2]:
class DeviceConfigs(BaseConfigs):
    use_cuda: bool = True
    cuda_device: int = 0

    device: any

Some configs can be calculated

In [3]:
@DeviceConfigs.calc(DeviceConfigs.device)
def cuda(c: DeviceConfigs):
    is_cuda = c.use_cuda and torch.cuda.is_available()
    if not is_cuda:
        return torch.device("cpu")
    else:
        if c.cuda_device < torch.cuda.device_count():
            return torch.device(f"cuda:{c.cuda_device}")
        else:
            logger.log(f"Cuda device index {c.cuda_device} higher than "
                       f"device count {torch.cuda.device_count()}", Text.warning)
            return torch.device(f"cuda:{torch.cuda.device_count() - 1}")

Configs classes can be inherited. This can be used to separate configs into module and it is quite neat when you want to inherit entire experiment setups and make a few modifications. 

In [4]:
class Configs(DeviceConfigs):
    model_size: int = 10
        
    model: any = 'cnn_model'

You can specify multiple config calculator functions. The function given by the string for respective attribute will be picked.

In [5]:
@Configs.calc(Configs.model)
def cnn_model(c: Configs):
    return c.model_size * 10

@Configs.calc(Configs.model)
def lstm_model(c: Configs):
    return c.model_size * 2

The experiment will calculate the configs.

In [6]:
conf = Configs()
conf.model = 'lstm_model'
experiment.create(name='test_configs')
experiment.calculate_configs(conf)
logger.inspect(model=conf.model)

In [7]:
experiment.start()